## Read File

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport


TRAIN_DIR = 'train_first'

CCBA_PATH = 'train_first/public_train_x_ccba_full_hashed.csv'
CDTX_PATH = 'train_first/public_train_x_cdtx0001_full_hashed.csv'
CUSTINFO_PATH = 'train_first/public_train_x_custinfo_full_hashed.csv'
DP_PATH = 'train_first/public_train_x_dp_full_hashed.csv'
REMIT_PATH = 'train_first/public_train_x_remit1_full_hashed.csv'
PDATE_PATH = 'train_first/public_x_alert_date.csv'
TDATE_PATH = 'train_first/train_x_alert_date.csv'
ANSWER_PATH = 'train_first/train_y_answer.csv'
SAMPLE_PATH = './sample_submission.csv'


ccba = pd.read_csv(CCBA_PATH)
cdtx = pd.read_csv(CDTX_PATH)
cinfo = pd.read_csv(CUSTINFO_PATH)
dp = pd.read_csv(DP_PATH)
remit = pd.read_csv(REMIT_PATH)
pdate = pd.read_csv(PDATE_PATH)
tdate = pd.read_csv(TDATE_PATH)
answer = pd.read_csv(ANSWER_PATH)
sample = pd.read_csv(SAMPLE_PATH)

/media/hd03/axot_data/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate Report

In [2]:
names = ['ccba', 'cdtx', 'custinfo', 'dp', 'remit', 'pdate', 'tdate', 'answer', 'sample']
datas = [ccba, cdtx, cinfo, dp, remit, pdate, tdate, answer, sample]
num_files = len(datas)

for i in range(num_files):
    print(f'{names[i]}: {datas[i].shape}')
    profile = ProfileReport(datas[i], minimal=True, title=names[i])
    profile.to_file(f'./data_report/{names[i]}.html', )

ccba: (59075, 10)


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 21.17it/s]


cdtx: (1043014, 5)


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 450.81it/s]


custinfo: (25751, 6)


...

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 789.59it/s]


answer: (23906, 2)


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 1164.44it/s]


sample: (3850, 2)


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 886.93it/s]


## Preprocess

In [1]:

import pickle

from sklearn.preprocessing import QuantileTransformer
import numpy as np
from easydict import EasyDict as edict
import pandas as pd
from tqdm import tqdm

from process_data.data_config import (DataSource, FeatureType,
                         CCBAConfig, CDTXConfig, DPConfig, REMITConfig, CUSTINFOConfig,
                         CONFIG_MAP)
from process_data.utils import load_yaml, save_yaml, save_pickle, load_pickle

### Map Date and target to `custinfo`

In [4]:
date = pd.concat([pdate, tdate], axis=0)
cinfo = cinfo.merge(date, on='alert_key', how='left')
cinfo = cinfo.merge(answer, on='alert_key', how='left')
cinfo

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE,date,sar_flag
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,365,NaN
1,352253,b212d14cb35676926682b2cf849e295d948888f556c07e...,1,2.0,98177.0,2,365,NaN
2,352254,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,1,19.0,2052922.0,7,365,NaN
3,352280,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,3,15.0,201906.0,5,365,NaN
4,352282,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,1,12.0,7450.0,5,365,NaN
...,...,...,...,...,...,...,...,...
25746,352123,b600c0720096c241f5aec16b096c6a353492eee8a5855a...,1,17.0,12207.0,2,364,0.0
25747,352124,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a...,1,17.0,259985.0,4,364,0.0
25748,352125,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3,364,0.0
25749,352128,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,3,19.0,21647.0,4,364,0.0


### Process Numerical and Categorical

In [5]:
def normalize(col):
    qt = QuantileTransformer(
        n_quantiles=10_000, 
        random_state=0, 
        subsample=min(5*10**5, len(col)),
        output_distribution='normal'
    )
    return qt.fit_transform(col)

def process_numerical(col):
    col = normalize(col)
    col = np.nan_to_num(col, nan=0)
    return col


def process_catgorical(col):
    col.fillna('NULL', inplace=True)
    map_dict = {v:i for i, v in enumerate(set(col.unique()))}
    col = col.map(map_dict)
    return col

In [6]:
datas = [
    (ccba, DataSource.CCBA), 
    (cdtx, DataSource.CDTX),
    (dp, DataSource.DP),
    (remit, DataSource.REMIT),
    (cinfo, DataSource.CUSTINFO),
]

num_cat_dict = {}

# process numerical and categorical and data_source
for data, data_source in datas:
    config = CONFIG_MAP[data_source]
    cols = data.columns
    numericals = []
    for col in cols:
        feature_type = getattr(config, col)
        if feature_type == FeatureType.NUMERICAL and col != 'sar_flag':
            numericals.append(col)
        elif feature_type == FeatureType.CATEGORICAL:
            data[col] = process_catgorical(data[col].copy())
            num_cat = data[col].nunique()
            if data_source not in num_cat_dict:
                num_cat_dict[data_source] = {}
            num_cat_dict[data_source][col] = num_cat

    if numericals:
        data[numericals] = process_numerical(data[numericals].copy())
    data['data_source'] = data_source

In [7]:
save_yaml(num_cat_dict, 'num_cat_dict.yml')

In [8]:
datas = [d[0] for d in datas]

In [9]:
datas_g = [d.groupby(by='cust_id') for d in datas]

In [21]:
def get_date(d):
    ds = d.data_source
    
    if ds == DataSource.CCBA:
        date = d.byymm
    elif ds == DataSource.CDTX:
        date = d.date
    elif ds == DataSource.DP:
        date = d.tx_date
    elif ds == DataSource.REMIT:
        date = d.trans_date
    elif ds == DataSource.CUSTINFO:
        date = d.date
    return date, ds


cust_ids = cinfo.cust_id.unique()
save_data = edict()
for cust_id in tqdm(cust_ids):
    # get all data from each group
    cust_data = []
    for d in datas_g:
        if not cust_id in d.groups:
            continue
        cust_data += d.get_group(cust_id).to_dict('records')
    for i in range(len(cust_data)):
        cust_data[i] = edict(cust_data[i])
    
    # sort by date
    cust_data.sort(key=get_date)
    
    # generate source list and target_mask
    source_list = []
    train_mask = []
    test_mask = []
    for i, c in enumerate(cust_data):
        ds = c.data_source
        source_list.append(ds)
        if ds != DataSource.CUSTINFO:
            pass
        elif np.isnan(c.sar_flag):
            test_mask.append(i)
        else:
            train_mask.append(i)
    
    # save data
    save_data[cust_id] = edict({
        'sources': source_list,
        'train_mask': train_mask,
        'test_mask': test_mask,
        'cust_data': cust_data,
    })


100%|██████████| 7708/7708 [03:23<00:00, 37.94it/s] 


### check num of train_mask, test_mask is correct

In [22]:
isna = cinfo.sar_flag.isna()
train_num = sum(~isna)
test_num = sum(isna)

train_num2 = 0
test_num2 = 0
for v in save_data.values():
    train_num2 += len(v.train_mask)
    test_num2 += len(v.test_mask)


print(train_num == train_num2)
print(test_num == test_num2)

True
True


In [26]:
save_pickle(save_data, 'cust_data.pkl')

### lenght distribution

In [27]:
# get len of cust_data of save_data
lens = []
for k, v in save_data.items():
    lens.append(len(v.sources))
pd.DataFrame(data=lens, columns=None).describe(percentiles=[.25, .5, .75, .9, .95, .99])

,0
count,7708.000000
mean,404.102880
std,2518.273585
min,1.000000
25%,29.000000
50%,138.000000
75%,399.000000
90%,848.900000
95%,1243.650000
99%,2747.650000


### check target_mask distribution

In [30]:
train_mask = []
test_mask = []
for v in save_data.values():
    train_mask += v.train_mask
    test_mask += v.test_mask

display(pd.DataFrame(data=train_mask, columns=['train']).describe(percentiles=np.linspace(0,1,11)))
display(pd.DataFrame(data=test_mask, columns=['test']).describe(percentiles=np.linspace(0,1,11)))

,train
count,23906.000000
mean,760.282272
std,3924.260433
min,0.000000
0%,0.000000
10%,7.000000
20%,30.000000
30%,69.000000
40%,121.000000
50%,196.000000


,test
count,1845.000000
mean,1364.484553
std,8040.953852
min,0.000000
0%,0.000000
10%,49.400000
20%,104.000000
30%,179.200000
40%,276.600000
50%,417.000000


In [2]:
data = load_pickle('/media/hd03/axot_data/sar/data/cust_data.pkl')
len(data)

7708

In [31]:
sars = set()
for cust_id, v in data.items():
    for idx in v.train_mask:
        if v.cust_data[idx].sar_flag == 1:
            sars.add(cust_id)
            break
len(sars)

233

In [32]:
num1 = []
for cust_id in sars:
    d = data[cust_id]
    tmp = 0
    for idx in d.train_mask:
        tmp += (d.cust_data[idx].sar_flag == 1)
    num1.append(tmp)
sum(num1)

234

In [33]:
# num_sar = []
num_len = []
num0 = []
num1 = []
for k, d in data.items():
    tmp0 = 0
    tmp1 = 0
    for idx in d.train_mask:
        tmp0 += d.cust_data[idx].sar_flag == 0
        tmp1 += d.cust_data[idx].sar_flag == 1
    num0.append(tmp0)
    num1.append(tmp1)
    num_len.append(len(d.cust_data))

In [34]:
df = pd.DataFrame({'num0': num0, 'num1':num1,'num_len': num_len})

In [35]:
df.describe()

,num0,num1,num_len
count,7708.000000,7708.000000,7708.000000
mean,3.071095,0.030358,404.102880
std,9.311131,0.172336,2518.273585
min,0.000000,0.000000,1.000000
25%,1.000000,0.000000,29.000000
50%,1.000000,0.000000,138.000000
75%,2.000000,0.000000,399.000000
max,236.000000,2.000000,135334.000000


In [36]:
df[df.num1>0].describe()

,num0,num1,num_len
count,233.000000,233.000000,233.000000
mean,2.802575,1.004292,1887.922747
std,8.547262,0.065512,8882.960612
min,0.000000,1.000000,21.000000
25%,0.000000,1.000000,295.000000
50%,0.000000,1.000000,641.000000
75%,0.000000,1.000000,1272.000000
max,72.000000,2.000000,124139.000000


In [44]:
# df1 = df[df.num1>0]
df[df.num_len == df.num_len.max()]

,num0,num1,num_len
136,68,0,135334


In [37]:
df[df.num1==0].describe()

,num0,num1,num_len
count,7475.000000,7475.0,7475.000000
mean,3.079465,0.0,357.851371
std,9.334339,0.0,2004.813683
min,0.000000,0.0,1.000000
25%,1.000000,0.0,27.000000
50%,1.000000,0.0,130.000000
75%,2.000000,0.0,380.000000
max,236.000000,0.0,135334.000000


In [41]:
mask_ids = []
for k, v in data.items():
    for i, idx in enumerate(v.train_mask):
        if i == 0:
            mask_ids.append(idx)
        else:
            mask_ids.append(idx-v.train_mask[i-1])
pd.DataFrame({'mask_ids': mask_ids}).describe(percentiles=np.arange(.9, 1.01, 0.01))

,mask_ids
count,23906.000000
mean,68.353049
std,486.181119
min,0.000000
50%,10.000000
90%,137.000000
91%,158.000000
92%,178.000000
93%,204.000000
94%,238.000000
